In [1]:
import pandas as pd

In [16]:
fee_rate = pd.read_csv('feerate.csv')

In [70]:
def feecal(value,fee_rate,rem):
    fee = 0
    g = value
    lor = 0
    
    rate = fee_rate[['upr',rem]]
    rate.columns = ['upr', 'rate']
    
    for fr in rate.itertuples():
        if value <= fr.upr:
            fee = fee + g * fr.rate
            break
        else:
            fee = fee + (fr.upr - lor) * fr.rate
            g = value - fr.upr
            lor = fr.upr
    return fee

In [54]:
g_test = [10000000,50000000,50050000,100000000,100050000,300000000,300050000,500000000,500050000,1000000000,1000050000]

In [71]:
for gt in g_test:
    print(str(gt) + '\t|\t' + str(feecal(gt,fee_rate,'const')))
#     print(feecal(gt,fee_rate,'const'))

10000000	|	132000.0
50000000	|	660000.0
50050000	|	660550.0
100000000	|	1210000.0
100050000	|	1210440.0
300000000	|	2970000.0
300050000	|	2970247.5
500000000	|	3960000.0
500050000	|	3960178.75
1000000000	|	5747500.0
1000050000	|	5747610.0
